In [ ]:
with open('/srv/scratch/z5297792/Sample/frontier_sample_data.pkl', 'rb') as file:
    frontier_sample_data = pickle.load(file)
the_frontier_eddies = list(frontier_sample_data.keys())
# Depths will be limited to COW depths
d_df = pd.DataFrame(columns=['T' + str(t) for t in range(30)])
for eddy in the_frontier_eddies:
    row = []
    for t in range(30):
        df = frontier_sample_data[eddy]['T' + str(t)]['COW']['x']
        row.append(df.isna().idxmax() if df.isna().any() else len(df))
    d_df.loc[eddy] = row

eddy_sub_data = {}

methods = ['VG', 'COW', 'LNAM']
for eddy in the_frontier_eddies:
    
    sub_data = {}
    for t in range(30):
        
        sub_data['T'+str(t)] = {}
    
        for method in methods:
    
            sub_data['T'+str(t)][method] = {}
    
            df = frontier_sample_data[eddy]['T'+str(t)][method].iloc[:d_df.loc[eddy]['T'+str(t)]]
            df = df.drop('TD', axis=1)
            
            # Interpolate to Density intervals
            rho = frontier_sample_data[eddy]['T'+str(t)]['rho']
            X, Y = np.meshgrid(frontier_sample_data[eddy]['T'+str(t)]['x'],
                               frontier_sample_data[eddy]['T'+str(t)]['y'])
            points = np.column_stack((X.ravel(), Y.ravel()))
            rhoc = []
            for k, (xc, yc) in enumerate(zip(df['x'], df['y'])):
                values = rho[:, :, k].ravel()
                target_point = np.array([[xc, yc]])
                rhoc.append(griddata(points, values, target_point, method='linear')[0])
            RHO_INT = 0.1  # Average d\rho/dz = 0.01
            new_rho = np.arange(np.floor(np.nanmin(rhoc) / RHO_INT) * RHO_INT,
                            np.ceil(np.nanmax(rhoc) / RHO_INT) * RHO_INT + RHO_INT,
                            RHO_INT)
            new_rho = [round(value, 2) for value in new_rho]
            rho_data = pd.DataFrame({
                'x': np.interp(new_rho, rhoc, df['x']),
                'y': np.interp(new_rho, rhoc, df['y']),
                'Depth': np.interp(new_rho, rhoc, df['Depth']),
                'rho': new_rho
            })
            x0 = rho_data.iloc[0]['x']
            y0 = rho_data.iloc[0]['y']
            rho_data['x'] = rho_data['x'] - x0
            rho_data['y'] = rho_data['y'] - y0
            rho_data['TD'] = np.hypot(rho_data['x'], rho_data['y'])
            
            # Interpolate to Depth intervals
            DEPTH_INT = 10
            new_depth = np.arange(0, -np.floor(df['Depth'].min()/DEPTH_INT) * DEPTH_INT + DEPTH_INT, DEPTH_INT)
            depth_data = pd.DataFrame({
                'x': np.interp(new_depth, -df['Depth'], df['x']),
                'y': np.interp(new_depth, -df['Depth'], df['y']),
                'Depth': -new_depth,
                'rho': np.interp(new_depth, -df['Depth'], rhoc),
            })
            x0 = depth_data.iloc[0]['x']
            y0 = depth_data.iloc[0]['y']
            depth_data['x'] = depth_data['x'] - x0
            depth_data['y'] = depth_data['y'] - y0
            depth_data['TD'] = np.hypot(depth_data['x'], depth_data['y'])
        
            sub_data['T'+str(t)][method]['rho_data'] = rho_data
            sub_data['T'+str(t)][method]['depth_data'] = depth_data
        if t % 10 == 0:
            print(t)
    eddy_sub_data[eddy] = sub_data
    print(eddy)
    del sub_data
